In [1]:
import json
import tqdm
from PIL import Image
import os
import glob
import numpy as np
from ultralytics import YOLO
import argparse
import torch

In [38]:
import json
import tqdm
from PIL import Image
import os
import glob
import numpy as np

base_path = "datasets/base"
label_path = "datasets/label"

# jpg와 json 파일 리스트를 가져옵니다.
jpg_files = [f for f in os.listdir(base_path) if f.endswith(".jpg")]
json_files = [f for f in os.listdir(label_path) if f.endswith(".json")]

# jpg와 json 파일 수가 같은지 확인합니다.
assert len(jpg_files) == len(json_files)

In [22]:
combined = list(zip(jpg_files, json_files))
random.shuffle(combined)
jpg_files, json_files = zip(*combined)

In [24]:
total_files = len(jpg_files)

train_size = int(0.7 * total_files)
val_size = int(0.2 * total_files)

train_jpgs = jpg_files[:train_size]
val_jpgs = jpg_files[train_size:train_size+val_size]
test_jpgs = jpg_files[train_size+val_size:]

train_jsons = json_files[:train_size]
val_jsons = json_files[train_size:train_size+val_size]
test_jsons = json_files[train_size+val_size:]

In [28]:
train_base_path = "datasets/train/base"
val_base_path = "datasets/val/base"
test_base_path = "datasets/test/base"

train_label_path = "datasets/train/label"
val_label_path = "datasets/val/label"
test_label_path = "datasets/test/label"

# 폴더 생성
for path in [train_base_path, val_base_path, test_base_path, train_label_path, val_label_path, test_label_path]:
    os.makedirs(path, exist_ok=True)

# 파일 이동
for f in train_jpgs:
    shutil.move(os.path.join(base_path, f), train_base_path)
for f in train_jsons:
    shutil.move(os.path.join(label_path, f), train_label_path)

for f in val_jpgs:
    shutil.move(os.path.join(base_path, f), val_base_path)
for f in val_jsons:
    shutil.move(os.path.join(label_path, f), val_label_path)

for f in test_jpgs:
    shutil.move(os.path.join(base_path, f), test_base_path)
for f in test_jsons:
    shutil.move(os.path.join(label_path, f), test_label_path)


In [31]:
import json
import os

def process_json(data, use_center_format=False):
    """ 주어진 JSON 데이터에서 shape_type이 bbox인 항목의 area_code와 points를 반환합니다. """
    output_str = ''
    image_width = data['description']['imageWidth']
    image_height = data['description']['imageHeight']
    
    for category, annotations in data['annotations'].items():
        for annotation in annotations:
            if annotation['shape_type'] == 'bbox':
                # bbox에서 중심점과 너비/높이를 계산
                left = annotation['points'][0]
                top = annotation['points'][1]
                width = annotation['points'][2]
                height = annotation['points'][3]


                #if use_center_format:
                x = (left + width/2) /image_width
                y = (top + height/2) /image_height
                width /= image_width
                height /= image_height
                points = [x, y, width, height]

                # else:
                #     # points 값을 스케일링
                #     points = [
                #         left / image_width,
                #         top / image_height,
                #         (left + width) / image_width,
                #         (top + height) / image_height
                #     ]

                code = int(annotation.get('area_code', annotation.get('PM_code', '')))
                code -= 13
                
                if code == 21:
                    continue

                if code > 21:
                    code -= 1
                #if code not in [0, 7, 15]:
                output_str += f"{code} {' '.join(map(str, points))}\n"
    return output_str

# JSON 파일이 있는 폴더 경로
source_folder = 'datasets/train/label'  # 이곳에 JSON 파일이 있는 폴더 경로를 입력하세요.

# 결과 텍스트 파일들을 저장할 폴더 경로
destination_folder = 'datasets/train/base'  # 이곳에 결과 텍스트 파일을 저장할 폴더 경로를 입력하세요.
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# 폴더 내의 모든 파일들을 순회
for filename in os.listdir(source_folder):
    if filename.endswith('.json'):
        json_path = os.path.join(source_folder, filename)
        
        # JSON 파일 읽기
        with open(json_path, 'r') as json_file:
            data = json.load(json_file)
            
            # JSON 데이터 처리
            result = process_json(data)
            
            # 결과를 지정된 경로의 text 파일로 저장
            txt_filename = filename.replace('.json', '.txt')
            txt_path = os.path.join(destination_folder, txt_filename)
            with open(txt_path, 'w') as txt_file:
                txt_file.write(result)

print("All JSON files have been processed and written to text files in the destination folder!")


All JSON files have been processed and written to text files in the destination folder!


In [1]:
import yaml

data = {'path' : '../ultralytics/datasets',
        'train' : 'D:/codestates/Team Project 2/ultralytics/datasets/train/base',
        'val' : 'D:/codestates/Team Project 2/ultralytics/datasets/val/base',
        'test' : 'D:/codestates/Team Project 2/ultralytics/datasets/test/base',
        'names' : ['오토바이', '오토바이_보행자도로 통행위반',
              '오토바이_안전모 미착용', '오토바이_무단횡단', '오토바이_신호위반', '오토바이_정지선위반',
              '오토바이_횡단보도 주행위반', '자전거', '자전거 캐리어', '자전거_보행자도로 통행위반',
              '자전거_안전모 미착용', '자전거_무단횡단', '자전거_신호위반', '자전거_정지선위반',
              '자전거_횡단보도 주행위반', '킥보드', '킥보드 캐리어', '킥보드_보행자도로 통행위반',
              '킥보드_안전모 미착용', '킥보드_무단횡단', '킥보드_신호위반', '킥보드_횡단보도 주행위반',
              '킥보드_동승자 탑승위반'],
        'nc' : 23 }

In [2]:
with open('sample.yaml', 'w') as f:
    yaml.dump(data, f)

with open('sample.yaml', 'r') as f:
    sample_yaml = yaml.safe_load(f)
    display(sample_yaml)

{'names': ['오토바이',
  '오토바이_보행자도로 통행위반',
  '오토바이_안전모 미착용',
  '오토바이_무단횡단',
  '오토바이_신호위반',
  '오토바이_정지선위반',
  '오토바이_횡단보도 주행위반',
  '자전거',
  '자전거 캐리어',
  '자전거_보행자도로 통행위반',
  '자전거_안전모 미착용',
  '자전거_무단횡단',
  '자전거_신호위반',
  '자전거_정지선위반',
  '자전거_횡단보도 주행위반',
  '킥보드',
  '킥보드 캐리어',
  '킥보드_보행자도로 통행위반',
  '킥보드_안전모 미착용',
  '킥보드_무단횡단',
  '킥보드_신호위반',
  '킥보드_횡단보도 주행위반',
  '킥보드_동승자 탑승위반'],
 'nc': 23,
 'path': '../ultralytics/datasets',
 'test': 'D:/codestates/Team Project 2/ultralytics/datasets/test/base',
 'train': 'D:/codestates/Team Project 2/ultralytics/datasets/train/base',
 'val': 'D:/codestates/Team Project 2/ultralytics/datasets/val/base'}

In [3]:
from ultralytics import YOLO
import argparse
# import torch
# device = torch.device("cuda")
model = YOLO('D:/codestates/Team Project 2/ultralytics/runs/detect/train75_v8s_60epoch/weights/best.pt')

In [15]:
import ultralytics

ultralytics.checks()

Ultralytics YOLOv8.0.160  Python-3.9.17 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 2060 SUPER, 8192MiB)
Setup complete  (16 CPUs, 31.9 GB RAM, 1044.8/1863.0 GB disk)


In [4]:
model.train(data='sample.yaml', epochs=1)

New https://pypi.org/project/ultralytics/8.0.167 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.160  Python-3.9.17 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 2060 SUPER, 8192MiB)
engine\trainer: task=detect, mode=train, model=D:/codestates/Team Project 2/ultralytics/runs/detect/train75_v8s_60epoch/weights/best.pt, data=sample.yaml, epochs=1, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1,

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from ultralytics import YOLO
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np


# Load a model
#model = YOLO('yolov8s.pt')  # load an official model
model = YOLO('D:/codestates/Team Project 2/ultralytics/runs/detect/train70_v8s_50epoch/weights/best.pt')  # load a custom model
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

In [2]:
print("mAP50-95:", metrics.box.map)
print("mAP50:", metrics.box.map50)
print("mAP75:", metrics.box.map75)

mAP50-95: 0.6186336582072937
mAP50: 0.7830307029122789
mAP75: 0.739344884661381


In [13]:
from ultralytics import YOLO
#
# Load a model
#model = YOLO('yolov8n.pt')  # load an official model
model = YOLO('D:/codestates/Team Project 2/ultralytics/runs/detect/train75_v8s_60epoch/weights/best.pt')  # load a custom model

# Predict with the model
results = model('D:/codestates/Team Project 2/ultralytics/sample2.mp4',stream=True)  # predict on an image

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    #masks = result.masks  # Masks object for segmentation masks outputs
    #keypoints = result.keypoints  # Keypoints object for pose outputs
    #probs = result.probs  # Probs object for classification outputs


video 1/1 (1/782) D:\codestates\Team Project 2\ultralytics\sample2.mp4: 384x640 (no detections), 16.0ms
video 1/1 (2/782) D:\codestates\Team Project 2\ultralytics\sample2.mp4: 384x640 (no detections), 14.0ms
video 1/1 (3/782) D:\codestates\Team Project 2\ultralytics\sample2.mp4: 384x640 (no detections), 15.0ms
video 1/1 (4/782) D:\codestates\Team Project 2\ultralytics\sample2.mp4: 384x640 (no detections), 15.0ms
video 1/1 (5/782) D:\codestates\Team Project 2\ultralytics\sample2.mp4: 384x640 (no detections), 14.0ms
video 1/1 (6/782) D:\codestates\Team Project 2\ultralytics\sample2.mp4: 384x640 (no detections), 14.0ms
video 1/1 (7/782) D:\codestates\Team Project 2\ultralytics\sample2.mp4: 384x640 (no detections), 17.0ms
video 1/1 (8/782) D:\codestates\Team Project 2\ultralytics\sample2.mp4: 384x640 (no detections), 15.0ms
video 1/1 (9/782) D:\codestates\Team Project 2\ultralytics\sample2.mp4: 384x640 (no detections), 15.0ms
video 1/1 (10/782) D:\codestates\Team Project 2\ultralytics\sam

In [1]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('D:/codestates/Team Project 2/ultralytics/runs/detect/train75_v8s_60epoch/weights/best.pt')

# Open the video file
video_path = "D:/codestates/Team Project 2/ultralytics/vidio/input/sample9.mp4"
cap = cv2.VideoCapture(video_path)

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('output_video9.avi', fourcc, fps, frame_size)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Save the annotated frame to the video
        out.write(annotated_frame)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture, video writer object and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 71.0ms
Speed: 6.0ms preprocess, 71.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.0ms
Speed: 4.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.0ms
Speed: 3.0ms preprocess, 15.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.8ms
Speed: 2.2ms preprocess, 13.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.0ms
Speed: 6.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.0ms
Speed: 3.0ms preprocess, 9.0ms infer

: 